In [1]:
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = torch.device("cuda")

In [106]:
import sys
sys.path.append('./huggingface_models/')
sys.path.append('./utils/')
# from sample_utils import *
# from inference_utils import *
# from leetcode_exec_utils import *
# from execution_utils import *
from self_training_utils import *

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
do_sample, temperature = False, 0.5

In [5]:
code_lang_dict, code_id_lang_dic = read_toked_code_dict(code_dict_path)
functions_pids_dict, functions_dict, functions_toked_dict, functions_detoked_dict = \
        get_all_functions_detok_from_cache(
            code_id_lang_dic, functions_pids_dict_path, functions_dict_path, 
            functions_toked_dict_path, functions_detoked_dict_path)
# function_json_dict, function_id_lang_dic = read_function_tok_file()
split_dict = load_split_dict()
test_list, val_list = get_eval_list(split_dict)

### XLCoST pipeline

In [7]:
lang_pairs = []
for lang1 in langs:
    for lang2 in langs:
        if lang2 != lang1:
            lang_pairs.append((lang1, lang2))

#### Select data and split

In [5]:
data_name = 'xlcost'
tag = "test"
function_data_path = home_path + "g4g/XLCoST_data/pair_data_notok_exec_function/"
data_code_dict_path = code_dict_path

#### Get original code execution performance

In [58]:
code_lang_dict, code_id_lang_dic = read_toked_code_dict(data_code_dict_path)
call_dict_gold = get_call_dict_gold(lang_pairs, function_data_path, code_id_lang_dic, data_name, tag)

#### Select model, get model predictions and do hypo filtering

In [18]:
sample_size = 5
model_type = "codet5"
exp_suffix = "_translation_exec_function/"

In [ ]:
label = ""
preds_lang_dict = get_preds_lang_dict(lang_pairs, model_type, device, None, None, function_data_path, 
                                      sample_size, temperature, data_name, tag, exp_suffix, label)
call_dict_hypo = get_call_dict_hypo(lang_pairs, preds_lang_dict, call_dict_gold, code_id_lang_dic, 
                                    model_type, data_name, tag, label)
hypo_filtering_dict = get_hypo_filtering_dict(lang_pairs, call_dict_gold, call_dict_hypo)

In [ ]:
# Eval codenet trained model
model_type = "plbart"
exp_suffix = "_translation_exec_codenet_small_function/"
new_langs = ["C++", "Java", "Python", "C#", "C"]
new_lang_pairs = []
for lang1 in new_langs:
    for lang2 in new_langs:
        if lang2 != lang1:
            new_lang_pairs.append((lang1, lang2))
label = "_exec_codenet" #_exec_codenet_small _exec_codenet_transfer_derail
# exp_suffix = "_translation_transfer_codenet_exec_function/"
preds_lang_dict = get_preds_lang_dict(new_lang_pairs, model_type, device, None, None, function_data_path, 
                                      sample_size, temperature, data_name, tag, exp_suffix, label)
call_dict_hypo = get_call_dict_hypo(new_lang_pairs, preds_lang_dict, call_dict_gold, code_id_lang_dic, 
                                    model_type, data_name, tag, label)
hypo_filtering_dict = get_hypo_filtering_dict(new_lang_pairs, call_dict_gold, call_dict_hypo)

In [140]:
# Eval codenet trained model
model_type = "plbart"
exp_suffix = "_translation_exec_codenet_function/"
new_langs = ["C++", "Java", "Python", "C#", "C"]
new_lang_pairs = []
for lang1 in new_langs:
    for lang2 in new_langs:
        if lang2 != lang1:
            new_lang_pairs.append((lang1, lang2))
label = "_exec_codenet" #_exec_codenet_small _exec_codenet_transfer_derail
# exp_suffix = "_translation_transfer_codenet_exec_function/"
preds_lang_dict = get_preds_lang_dict(new_lang_pairs, model_type, device, None, None, function_data_path, 
                                      sample_size, temperature, data_name, tag, exp_suffix, label)
call_dict_hypo = get_call_dict_hypo(new_lang_pairs, preds_lang_dict, call_dict_gold, code_id_lang_dic, 
                                    model_type, data_name, tag, label)
hypo_filtering_dict = get_hypo_filtering_dict(new_lang_pairs, call_dict_gold, call_dict_hypo)
exec_rate_dict1 = get_exec_rate_dict(hypo_filtering_dict)

C++ Java
432 103 296 831
C++ Python
300 202 264 766
C++ C#
466 77 282 825
C++ C
20 10 10 40
Java C++
386 98 347 831
Java Python
307 193 279 779
Java C#
547 37 258 842
Java C
16 8 15 39
Python C++
322 148 296 766
Python Java
373 113 293 779
Python C#
429 100 252 781
Python C
4 17 10 31
C# C++
418 106 301 825
C# Java
580 39 223 842
C# Python
321 234 226 781
C# C
18 10 12 40
C C++
32 3 5 40
C Java
22 6 11 39
C Python
7 15 9 31
C C#
25 5 10 40


#### Eval codex

In [131]:
model_type = "codex"
label = ""
with open(cached_path + 'merged_filtered_codex_dict.pkl', 'rb') as infile:
    merged_filtered_codex_dict = pickle.load(infile)
codex_preds_lang_dict, codex_pids_dict = get_preds_lang_dict_codex(merged_filtered_codex_dict, call_dict_gold)
lang_pairs = list(codex_preds_lang_dict.keys())
call_dict_hypo = get_call_dict_hypo_codex(lang_pairs, codex_preds_lang_dict, codex_pids_dict, 
                                    call_dict_gold, code_id_lang_dic, 
                                    model_type, data_name, tag, label)
hypo_filtering_dict = get_hypo_filtering_dict_pids(lang_pairs, call_dict_gold, call_dict_hypo, codex_pids_dict)
exec_rate_dict = get_exec_rate_dict(hypo_filtering_dict)

 20%|██        | 91/451 [00:00<00:00, 907.17it/s]

Java Python


100%|██████████| 451/451 [00:00<00:00, 740.98it/s]


 22%|██▏       | 31/138 [00:00<00:00, 302.98it/s]

Python error 314 0.6962305986696231
Python timeout 1 0.0022172949002217295
Python empty 4 0.008869179600886918
Python other 0 0.0
Python good 132 0.2926829268292683
Python Java


...

100%|██████████| 39/39 [00:00<00:00, 479.87it/s]

Java error 21 0.25301204819277107
Java timeout 0 0.0
Java empty 0 0.0
Java other 0 0.0
Java good 62 0.7469879518072289
C++ C


100%|██████████| 39/39 [00:00<00:00, 458.20it/s]

C error 9 0.23076923076923078
C timeout 0 0.0
C empty 0 0.0
C other 0 0.0
C good 30 0.7692307692307693
Java C



C error 8 0.20512820512820512
C timeout 0 0.0
C empty 0 0.0
C other 0 0.0
C good 31 0.7948717948717948
Java Python
314 46 91 451
Python Java
74 5 59 138
C++ Python
96 32 61 189
Python C++
87 9 48 144
C Python
12 6 10 28
Python C
10 2 19 31
Java C#
68 0 62 130
C# Java
21 4 58 83
C++ C
9 3 27 39
Java C
8 2 29 39


In [132]:
model_type = "plbart"
exp_suffix = "_translation_exec_codenet_small_function/"
label ="_exec_codenet_small"
# lang_pairs = list(codex_preds_lang_dict.keys())
preds_lang_dict = get_preds_lang_dict(lang_pairs, model_type, device, None, None, function_data_path, 
                                      sample_size, temperature, data_name, tag, exp_suffix, label)
call_dict_hypo = get_call_dict_hypo(new_lang_pairs, preds_lang_dict, call_dict_gold, code_id_lang_dic, 
                                    model_type, data_name, tag, label)
hypo_filtering_dict = get_hypo_filtering_dict_pids(lang_pairs, call_dict_gold, call_dict_hypo, codex_pids_dict)
exec_rate_dict_pcn_small = get_exec_rate_dict(hypo_filtering_dict)

Java Python
209 121 121 451
Python Java
70 28 40 138
C++ Python
109 42 38 189
Python C++
90 20 34 144
C Python
8 14 6 28
Python C
10 15 6 31
Java C#
128 0 2 130
C# Java
83 0 0 83
C++ C
21 10 8 39
Java C
19 6 14 39


In [133]:
model_type = "plbart"
# exp_suffix = "_translation_exec_codenet_small_function/"
# label ="_exec_codenet_small"
exp_suffix = "_translation_exec_codenet_function/"
label ="_exec_codenet" 

# lang_pairs = list(codex_preds_lang_dict.keys())
preds_lang_dict = get_preds_lang_dict(lang_pairs, model_type, device, None, None, function_data_path, 
                                      sample_size, temperature, data_name, tag, exp_suffix, label)
call_dict_hypo = get_call_dict_hypo(new_lang_pairs, preds_lang_dict, call_dict_gold, code_id_lang_dic, 
                                    model_type, data_name, tag, label)
hypo_filtering_dict = get_hypo_filtering_dict_pids(lang_pairs, call_dict_gold, call_dict_hypo, codex_pids_dict)
exec_rate_dict_pcn = get_exec_rate_dict(hypo_filtering_dict)

Java Python
184 106 161 451
Python Java
75 15 48 138
C++ Python
85 45 59 189
Python C++
60 29 55 144
C Python
7 13 8 28
Python C
4 17 10 31
Java C#
88 5 37 130
C# Java
67 3 13 83
C++ C
20 10 9 39
Java C
16 8 15 39


#### Get execution rate

In [142]:
print_exec_results(exec_rate_dict, new_langs=new_langs)

\t\t32.28\t\t69.23\t
\t\t20.18\t47.69\t74.36\t
33.33\t42.75\t\t\t61.29\t
\t69.88\t\t\t\t
\t\t35.71\t\t\t


In [141]:
print_exec_results(exec_rate_dict1, new_langs=new_langs)

\t35.62\t34.46\t34.18\t25.0\t
41.76\t\t35.82\t30.64\t38.46\t
38.64\t37.61\t\t32.27\t32.26\t
36.48\t26.48\t28.94\t\t30.0\t
12.5\t28.21\t29.03\t25.0\t\t


In [139]:
# exec_rate_dict
from sample_utils import *
print_exec_results(exec_rate_dict_pcn, new_langs=new_langs)

\t\t31.22\t\t23.08\t
\t\t35.7\t28.46\t38.46\t
38.19\t34.78\t\t\t32.26\t
\t15.66\t\t\t\t
\t\t28.57\t\t\t


#### Eval new experiment

In [ ]:
exp_suffix = "_translation_exec_codenet_function/"
lang_pairs = [('Java', "Python"), ('Python', "Java"), ('Java', "C#"), ('C#', "Java")]
preds_lang_dict = get_preds_lang_dict(lang_pairs, model_type, device, None, None, function_data_path, 
                                      sample_size, temperature, data_name, tag, exp_suffix, label="haha")

In [27]:
# Golden results are in result_id_dict_gold
(map_dict, reverse_map_dict), pids, programs_gold, \
(result_id_dict_gold,  result_key_dict_gold, error_type_dict_gold) = call_dict_gold[(lang1, lang2)]

In [29]:
# passed_hypo_dict contains hypo that matches the output of golden.
lang1 = "Java"
lang2 = "Python"
preds = preds_lang_dict[(lang1, lang2)]
pids, programs, program_id_dict, program_dict, (result_id_dict, result_key_dict, error_type_dict)\
    = call_dict_hypo[(lang1, lang2)]
buggy_pids, failed_test_pids, passed_hypo_dict = hypo_filtering_dict[(lang1, lang2)]

#### Repeated sampling

In [ ]:
num_iterations = 2
is_eval = True
repeated_sampling_dict = get_repeated_sampling_dict(lang_pairs, model_type, device, 
                                         None, None, function_data_path, code_id_lang_dic, 
                                         data_name, 'val', exp_suffix, 
                                         is_eval, num_iterations, sample_size, temperature)

In [28]:
with open(cached_path + "plbart_test_xlcost_repeated_sampling_dict.pkl", 'wb') as outfile:
    pickle.dump(repeated_sampling_dict, outfile)

In [10]:
with open(cached_path + "plbart_test_xlcost_repeated_sampling_dict.pkl", 'rb') as infile:
    repeated_sampling_dict = pickle.load(infile)

In [11]:
lang1 = "Java"
lang2 = "Python"
buggy_pids, failed_test_pids, good_hypo_dict = repeated_sampling_dict[(lang1, lang2)]

In [15]:
print(len(buggy_pids), len(failed_test_pids), len(good_hypo_dict))

164 188 427


### Leetcode Pipeline

In [ ]:
lang_pairs = []
for lang1 in tri_langs:
    for lang2 in tri_langs:
        if lang2 != lang1:
            lang_pairs.append((lang1, lang2))

In [ ]:
data_name = 'leetcode'
function_data_path = leetcode_pair_data_path + "pair_data_notok_exec_leetcode/"
data_code_dict_path = leetcode_code_dict_path
code_lang_dict, code_id_lang_dic = read_toked_code_dict(data_code_dict_path, tri_langs)
# leetcode has to be evaluated on testcases.
# call_dict_gold = get_call_dict_gold(lang_pairs, function_data_path, code_id_lang_dic, data_name, tag)

In [ ]:
results_dict = {}
call_dict = {}
for lang in tri_langs:
    print(lang)
    programs, program_id_dict, pids = prepare_exec_program(code_id_lang_dic, lang)
    call_dict[lang] = programs, program_id_dict, pids
    print(len(programs))
    lang_results = p_map(file_executors[lang], programs)
    results_dict[lang] = lang_results

In [ ]:
# leetcode pairwise is only a small part of leetcode. The rest of it is unaligned.
# try to get the single pids in eahc lang
leetcode_split_dict = get_leetcode_split(None)

In [10]:
sample_size = 5
model_type = "plbart"
exp_suffix = "translation_leetcode_exec_function/" #translation_transfer_leetcode_exec_function
preds_lang_dict = get_preds_lang_dict(lang_pairs, model_type, device, function_data_path, 
                                      data_name, tag, exp_suffix)
call_dict_hypo = get_call_dict_hypo(lang_pairs, preds_lang_dict, call_dict_gold, code_id_lang_dic, 
                                    model_type, data_name, tag)

/home/mingzhu/CodeModel/CodeGen_cwd/cached_files/plbart_test_xlcost_call_dict_hypo.pkl


In [17]:
with open(cached_path + "plbart_xlcost_call_dict_hypo.pkl", 'wb') as outfile:
    pickle.dump(call_dict_hypo, outfile)

In [ ]:
hypo_filtering_dict = get_hypo_filtering_dict(lang_pairs, call_dict_gold, call_dict_hypo)

### Pipeline for single language pair

In [ ]:
data_name = 'xlcost'
tag = "test"
function_data_path = home_path + "g4g/XLCoST_data/pair_data_notok_exec_function/"
data_code_dict_path = code_dict_path
lang1 = "Java"
lang2 = "C#" 
code_lang_dict, code_id_lang_dic = read_toked_code_dict(data_code_dict_path)

In [111]:
model_type = "plbart"
model, tokenizer, decoder_sid = get_model_by_name(model_type, device, lang1, lang2)

In [38]:
test_file, args = prep_eval_args(model_type, tokenizer, lang1, lang2, function_data_path)
args.eval_batch_size = 32
eval_examples, eval_features = get_eval_examples_from_file(test_file, tokenizer, args)
eval_dataloader = get_eval_dataloader(eval_features, args.eval_batch_size)

In [10]:
code_string = "static int minSum ( int A [ ] , int N ) { HashMap < Integer , Integer > mp = new HashMap < Integer , Integer > ( ) ; int sum = 0 ; for ( int i = 0 ; i < N ; i ++ ) { sum += A [ i ] ; if ( mp . containsKey ( A [ i ] ) ) { mp . put ( A [ i ] , mp . get ( A [ i ] ) + 1 ) ; } else { mp . put ( A [ i ] , 1 ) ; } } int minSum = Integer . MAX_VALUE ; for ( Map . Entry < Integer , Integer > it : mp . entrySet ( ) ) { minSum = Math . min ( minSum , sum - ( it . getKey ( ) * it . getValue ( ) ) ) ; } return minSum ; }"
batch = get_eval_tensors(code_string, tokenizer, args)
pred = get_generation_demo(batch, model, tokenizer, args, device, False, decoder_sid)


In [33]:
torch.cuda.empty_cache()

### Beam search (or greedy decoding) with single return sequence

In [ ]:
do_sample = False
preds, dev_bleu, xmatch = generation_single(eval_examples, eval_dataloader, 
                                            model, tokenizer, args, device, 
                                            do_sample, temperature)

### Sampling with single return sequence

In [15]:
do_sample = True
preds, dev_bleu, xmatch = generation_single(eval_examples, eval_dataloader, 
                                            model, tokenizer, args, device, 
                                            do_sample, temperature, )

100%|██████████| 8/8 [00:36<00:00,  4.62s/it]


  bleu-4 = 71.13 
  xMatch = 7.0312 
  ********************


### Sampling with multiple return sequences

In [ ]:
preds, accs, dev_bleus, dev_bleu, xmatch = generation_multiple(eval_examples, eval_dataloader, 
                                                             model, tokenizer, args, device, 
                                                             decoder_sid, 5, 0.5)

In [ ]:
num_samples = 5
out_fn = get_out_fn(True, temperature, args.beam_size)
preds = get_sample_generations(eval_dataloader, model, tokenizer, args, device, 
                               decoder_sid, num_samples, temperature)
accs, dev_bleus, dev_bleu, xmatch = eval_bleu_samples(eval_examples, args, preds, out_fn)

#### Try sampling on codebert (failed)

In [ ]:
hypos=[]
pred_ids = []
for batch in tqdm(eval_dataloader,total=len(eval_dataloader)):
    batch = tuple(t.to(device) for t in batch)
    source_ids,source_mask= batch                  
    with torch.no_grad():
        preds = model(source_ids=source_ids, source_mask=source_mask, )

In [30]:
from run import *

In [32]:
lang1 = "Java"
lang2 = "Python"

In [36]:
model_type = "codebert"
model_name_or_path = model_name_dict[model_type]
model_path = func_model_path + model_type + "/" + model_type + exp_suffix
load_model_path_prefix = model_path + lang1 + "-" + lang2 + '/'
load_model_path = load_model_path_prefix + "checkpoint-best-bleu/pytorch_model.bin"
config_class, model_class, tokenizer_class = MODEL_CLASSES[model_type]
config = config_class.from_pretrained(model_name_or_path)
tokenizer = tokenizer_class.from_pretrained(model_name_or_path)
decoder_sid = tokenizer.eos_token_id
if model_type == "codebert":
    encoder = model_class.from_pretrained(model_name_or_path)    
    decoder_layer = nn.TransformerDecoderLayer(d_model=config.hidden_size, nhead=config.num_attention_heads)
    decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
    model=Seq2Seq(encoder=encoder,decoder=decoder,config=config,
                  sos_id=tokenizer.cls_token_id,eos_id=tokenizer.sep_token_id)
else:
    model = model_class.from_pretrained(model_name_or_path)
if model_type != "plbart":
    decoder_sid = None
model.load_state_dict(torch.load(load_model_path))
model.eval()
model.to(device)
print()

In [41]:
args.model_type = 'roberta'

In [20]:
# get preds_lang_dict for codebert
model_type = "codet5"
model_path = func_model_path + model_type + "/" + model_type + exp_suffix
preds_lang_dict = {}
for lang1, lang2 in lang_pairs:
    lang_pair_path = model_path + lang1 + "-" + lang2 + '/'
    hypo_path = lang_pair_path + "test_0.output"
    with open(hypo_path) as infile:
        lines = infile.readlines()
    preds = [[x.strip()] for x in lines]
    preds_lang_dict[(lang1, lang2)] = preds
label = "beam"
preds_lang_dict_path = \
    cached_path + model_type + "_" + tag + "_" + data_name + "_preds_lang_dict" + label + ".pkl"
with open(preds_lang_dict_path, 'wb') as outfile:
    pickle.dump(preds_lang_dict, outfile)

### Check compilation rate

In [115]:
map_dict, reverse_map_dict = get_target_map_dict(lang1, lang2, function_data_path)
pids = [map_dict[x] for x in range(len(map_dict))]

In [116]:
programs_gold, lang_results_gold, result_dicts = exec_gold(map_dict, lang2, code_id_lang_dic)
result_id_dict_gold, result_key_dict_gold, error_type_dict_gold = result_dicts
error_type_dict_gold


C# error 0 0.0
C# timeout 0 0.0
C# empty 0 0.0
C# other 0 0.0
C# good 842 1.0


In [119]:
programs, program_id_dict, program_dict = prep_exec_hypo(preds, pids, code_id_lang_dic, lang2, model_type)

100%|██████████| 842/842 [00:11<00:00, 74.61it/s]


In [121]:
lang_results = p_map(file_executors[lang2], programs)

In [ ]:
result_dict = {lang2:lang_results}
result_type_dict = show_result_summary(result_dict)
result_id_dict, result_key_dict, error_type_dict = result_mapping(lang_results, program_id_dict, pids, lang2)
print(error_type_dict)
buggy_pids, failed_test_pids, passed_hypo_dict = hypo_filtering(result_id_dict, 
                                                                result_id_dict_gold, 
                                                                result_key_dict, 
                                                                program_dict)
print(len(buggy_pids),len(failed_test_pids), len(passed_hypo_dict), len(pids))
fail_pids = buggy_pids + failed_test_pids

In [ ]:
a = file_executors[lang2](b)
print(a)

In [ ]:
print_src("".join(code_id_lang_dic['Java']['3096']['program_pieces']))

In [ ]:
b = detok_format(program_dict['3096'][1], file_detokenizers[lang2])
print_src(b)

In [ ]:
for idx, result_keys in result_key_dict.items():
    for i, key in enumerate(result_keys):
        if key == "error":
            error = result_id_dict[idx][i]
            if "a function-definition is not allowed" in error:
                print(idx, i, error)
                break

In [ ]:
# 312 572 209 363 884 sample_size=1 plbart temper=0.5
# 227 657 247 410 884 sample_size=5 plbart temper=0.5
# 196 688 256 432 884 sample_size=10 plbart temper=0.5

In [ ]:
buggy_pids, failed_test_pids, passed_hypo_dict = sampling_filtering(pids, eval_examples, eval_features, 
                     function_id_lang_dic, reverse_map_dict, lang,
                    model, tokenizer, args, device, decoder_sid, 10, 0.5)

In [ ]:
buggy_pids, failed_test_pids, good_hypo_dict = repeated_sampling(pids, eval_examples, eval_features, 
                                                     function_id_lang_dic, reverse_map_dict, lang,
                                                     model, tokenizer, args, device, decoder_sid, 
                                                     5, temperature, 4)

In [ ]:
print_src(passed_hypo_dict['7390'][0])

In [ ]:
print_src(function_id_lang_dic[lang2]['7390']['program_formatted'])

In [ ]:
# filter out output that's binary (this is why multiple testcase is needed)
# channel in more data from other datasets

In [ ]:
pid = '1535'
# function_id_lang_dic[lang2][key]['code_dict']
key = 2
print_src(program_dict[pid][key])
print(result_id_dict[pid][key])

In [87]:
program = "".join(function_id_lang_dic[lang2][pid]['code_dict']['program_pieces']).replace(
    target_function_place_holder, program_dict[pid][key])
print_src(program)
b = run_exec_python(program)

1 def reversed ( string ) :
2     if string == None or ( len ( string ) <= 1 ) :
3         print string
4     else :
5         print string [ len ( string ) - 1 ]
6         string = string [ : len ( string ) - 1 ]
7         string [ len ( string ) - 1 ] = string [ len ( string ) - 1 ]
8 
9 string = "Geeks for Geeks"
10 reverse(string)
